In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import missingno as msno


warnings.filterwarnings("ignore")
sns.set_theme(style="whitegrid", palette="Spectral")

In [118]:
# importing data
costs_df = pd.read_csv('data/costs.csv')
profiles_df = pd.read_csv('data/profiles.csv')
verification_df = pd.read_csv('data/verification.csv')


# Data in a glance 

In [119]:
costs_df.head()

,country,whatsapp_usd,sms_usd
0,AC,0.043,0.107000
1,AD,0.043,0.130000
2,AE,0.026,0.063066
3,AF,0.040,0.180605
4,AG,0.043,0.108000


In [120]:
costs_df.shape

(228, 3)

In [121]:
costs_df.describe()

,whatsapp_usd,sms_usd
count,228.000000,228.000000
mean,0.043276,0.132274
std,0.005594,0.080146
min,0.026000,0.007600
25%,0.043000,0.068000
50%,0.043000,0.120000
75%,0.043000,0.171127
max,0.086000,0.440000


In [122]:
profiles_df.head()

,userID,gender,dob,country
0,mpr_2fOzSxcEMql0TQkeK2vGCuJBu9K,M,"25 Oct, 2005, 00:00",CA
1,mpr_2fIcL4JraEe6OMdD4xvfVIZiAx8,M,"10 Jun, 1978, 00:00",CA
2,mpr_2fHMFzUtfSXQORtB5anUIjuNQso,F,"15 Feb, 2002, 00:00",GB
3,mpr_2fHaKh68oUhLyEZwztVzaCHefQ9,F,"25 Aug, 2001, 00:00",FR
4,mpr_2fIJtYAsOQ8AqU402d8j5yEFLqt,M,"18 Apr, 1990, 00:00",FR


In [123]:
profiles_df.shape

(18424, 4)

In [124]:
profiles_df.describe()

,userID,gender,dob,country
count,18424,18424,18424,18421
unique,18424,2,7369,159
top,mpr_2fOzSxcEMql0TQkeK2vGCuJBu9K,M,"19 Apr, 2000, 00:00",MA
freq,1,11387,63,2037


In [125]:
verification_df.head()

,userID,group,method,verified
0,mpr_2fHOBr0Nw6mNM2fPIKMeSyKUP0e,B,Sms,0
1,mpr_2fHOmoydcZfjF0UdFEmDQQifYK1,C,Sms,1
2,mpr_2fHRBhLrWEOcfjjblwatVXbnxoN,B,Sms,1
3,mpr_2fHQGGdXwbv8902hxC9C1dt1ThN,A,Sms,1
4,mpr_2fHSJ6Yyz6VC06GVQFjXH9q7s6m,A,Sms,1


In [126]:
verification_df.shape

(18424, 4)

# Data Validation

In [127]:
missing_values = profiles_df.isnull().sum()
missing_values

userID     0
gender     0
dob        0
country    3
dtype: int64

In [128]:
profiles_df[profiles_df['country'].isnull()]

,userID,gender,dob,country
308,mpr_2fMRl7NrXhXuM8ahHKpUYYoq1qZ,M,"20 Dec, 2003, 00:00",NaN
10810,mpr_2fF489IMIBWqhDlQYzIzGrolOhM,F,"16 Apr, 1988, 00:00",NaN
11674,mpr_2fN0uUj3O9ywDBGrjtW1mNoU3si,M,"8 May, 1983, 00:00",NaN


In [129]:
print(verification_df[verification_df['userID'] == 'mpr_2fF489IMIBWqhDlQYzIzGrolOhM'])
print(verification_df[verification_df['userID'] == 'mpr_2fN0uUj3O9ywDBGrjtW1mNoU3si'])
print(verification_df[verification_df['userID'] == 'mpr_2fMRl7NrXhXuM8ahHKpUYYoq1qZ'])


                               userID group method  verified
3522  mpr_2fF489IMIBWqhDlQYzIzGrolOhM     A    Sms         0
                                userID group method  verified
12778  mpr_2fN0uUj3O9ywDBGrjtW1mNoU3si     A    Sms         0
                                userID group    method  verified
14921  mpr_2fMRl7NrXhXuM8ahHKpUYYoq1qZ     B  Whatsapp         1


In [130]:
profiles_df["dob"] = pd.to_datetime(profiles_df['dob']).dt.strftime("%Y-%m-%d")
profiles_df['dob'] = pd.to_datetime(profiles_df['dob'], errors='coerce')
profiles_df["dob"].head()

0   2005-10-25
1   1978-06-10
2   2002-02-15
3   2001-08-25
4   1990-04-18
Name: dob, dtype: datetime64[ns]

In [131]:
merged_df = pd.merge(profiles_df, verification_df, on='userID', how='inner')
merged_df.shape

(18424, 7)

In [ ]:
melted_costs_df = costs_df.melt(id_vars=['country'], var_name='method', value_name='cost')

In [ ]:
# change 'whatsapp and sms _used' to 'whatsapp' and 'Sms' in 'method' column
melted_costs_df['method'] = melted_costs_df['method'].replace({'whatsapp_usd': 'Whatsapp'})
melted_costs_df['method'] = melted_costs_df['method'].replace({'sms_usd': 'Sms'})

In [139]:
merged_df = pd.merge(merged_df, melted_costs_df, on= ['method','country'], how='left')

# Feature engineering

In [ ]:
merged_df['age'] = 2024 - merged_df['dob'].dt.year

merged_df['age_group'] = pd.cut(merged_df['age'], bins=[17, 22, 27, 32, 44, 54,130])

# Data Visualisation 

In [ ]:
sns.countplot(x='verified', data=merged_df)
plt.xlabel('Verified Status')
plt.ylabel('Count')
plt.bar(merged_df['verified'].value_counts().index,
        merged_df['verified'].value_counts().values)

In [ ]:
merged_df['verified'].value_counts(normalize=True)

In [ ]:
plt.figure(figsize=(25,5))

top_countries = merged_df['country'].value_counts().nlargest(10).index
filtered_df = merged_df[merged_df['country'].isin(top_countries)]

# plot
sns.countplot(
    x='country',
    data=filtered_df,
    order=top_countries  # ensures correct order
)

plt.xlabel('Country')
plt.ylabel('Count')
plt.title('Top 10 Countries by Count')


In [ ]:
sns.countplot(x='gender', data=merged_df)
plt.xlabel('gender')
plt.ylabel('Count')
plt.bar(merged_df['gender'].value_counts().index,
        merged_df['gender'].value_counts().values)

In [ ]:
merged_df['gender'].value_counts(normalize=True)

In [ ]:
sns.histplot(merged_df['age'], kde=True)
plt.xlabel('Age')

In [ ]:
# plot age group counts
age_counts = merged_df['age_group'].value_counts().sort_index()
age_counts_df = age_counts.reset_index()
age_counts_df.columns = ['age_group', 'count']

sns.barplot(x='age_group', y='count', data=age_counts_df)
plt.title('Count by age group')

## Group

In [ ]:
merged_df['group'].value_counts()

In [ ]:
group_verified = merged_df.groupby(['group', 'verified']).size().unstack()
group_verified_percent = group_verified.div(group_verified.sum(axis=1), axis=0) * 100
print(group_verified_percent.round(2))

## Method

In [ ]:
group_method= merged_df.groupby(['group', 'method']).size().unstack()
group_method_percent = group_method.div(group_method.sum(axis=1), axis=0) * 100
print(group_method_percent.round(2))

In [ ]:
group_method

In [ ]:
group_method_verified = merged_df.groupby(['group', 'method', 'verified']).size().unstack()
group_method_verified_percent = group_method_verified.div(group_method_verified.sum(axis=1), axis=0) * 100
print(group_method_verified_percent.round(2))

In [ ]:
group_method_verified

## Gender

In [ ]:
gender_verified = merged_df.groupby(['gender', 'verified']).size().unstack()
gender_verified_percent = gender_verified.div(gender_verified.sum(axis=1), axis=0) * 100
print(gender_verified_percent.round(2))

In [ ]:
gender_group_verified = merged_df.groupby(['gender', 'group', 'verified']).size().unstack()
gender_group_verified_percent = gender_group_verified.div(gender_group_verified.sum(axis=1), axis=0) * 100
print(gender_group_verified_percent.round(2))

## Age

In [ ]:
age_group_verified = merged_df.groupby(['age_group', 'verified']).size().unstack()
age_group_verified_percent = age_group_verified.div(age_group_verified.sum(axis=1), axis=0) * 100
print(age_group_verified_percent.round(2))

In [ ]:
age_group_verified

In [ ]:
age_group_group_verified = merged_df.groupby(['age_group','group', 'verified']).size().unstack()
age_group_group_verified_percent = age_group_group_verified.div(age_group_group_verified.sum(axis=1), axis=0) * 100
print(age_group_group_verified_percent.round(2))


In [ ]:
age_group_group_method = merged_df.groupby(['age_group','group', 'method']).size().unstack()
age_group_group_method_percent = age_group_group_method.div(age_group_group_verified.sum(axis=1), axis=0) * 100
print(age_group_group_method_percent.round(2))

## Cost

In [147]:
method_cost = merged_df[merged_df['group'] !='A'].groupby(['method'])['cost'].mean()
method_cost

method
Sms         0.094256
Whatsapp    0.043557
Name: cost, dtype: float64

In [148]:
method_cost = merged_df[merged_df['group'] !='A'].groupby(['method'])['cost'].sum()
method_cost

method
Sms         704.090471
Whatsapp    199.099000
Name: cost, dtype: float64

In [149]:
method_cost = merged_df[merged_df['group'] !='A'].groupby(['method'])['cost'].count()
method_cost

method
Sms         7470
Whatsapp    4571
Name: cost, dtype: int64